In [1]:
from google.cloud import bigquery
from google.cloud.bigquery import job
import pandas as pd
import os
from tqdm import tqdm
PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)

c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [5]:
query = """
WITH prod as (
  SELECT Event_Date,User_ID, JSON_EXTRACT_ARRAY(Semantic_Event_Properties, '$.products') AS products
  FROM `ballosodeuk.airbridge_lake.app`
  WHERE  
    TIMESTAMP_TRUNC(_PARTITIONTIME, DAY) >= TIMESTAMP("2024-04-29") AND
    Event_Category = 'Ad Impression (App)' AND Event_Label = '홈_상단_캐시버튼' AND Event_Action = '직광고_쿠팡' And
    Semantic_Event_Properties is not Null AND
    Event_Date between "2024-04-11" AND "2024-04-24"
    ) 
    ,

click as (
  select distinct User_ID
  from `ballosodeuk.airbridge_lake.app`
  where 
    TIMESTAMP_TRUNC(_PARTITIONTIME, DAY) >= TIMESTAMP("2024-04-29") AND 
    Event_Category = 'Ad Click (App)' AND
    Event_Date between "2024-04-11" AND "2024-04-24"
)
  
select p.Event_Date, p.User_ID,
  ARRAY_TO_STRING(ARRAY(
  select JSON_EXTRACT_SCALAR(products, '$.name')
  from UNNEST(p.products) As products
  ),',') as product_names,
  IF(p.Event_Date between "2024-04-11" AND "2024-04-17",'before','after') as period
from prod p
join 
  click c on p.User_ID = c.User_ID

"""

In [9]:
query = """
SELECT DISTINCT User_ID
FROM `ballosodeuk.airbridge_lake.app_2024` 
WHERE Event_Date Between "2024-04-26" AND "2024-05-02"
  ANd Event_Category = "Sign-up (App)"
  AND Channel = 'invite_code'
"""

In [10]:
job_config = bigquery.QueryJobConfig()
job_config.use_query_cache = False
job_config.allow_large_results = True

In [11]:
query_job = bq.query(query,job_config=job_config)

In [ ]:
# 쿼리 실행 및 결과 저장
# query_job = bq.query(query)

# # 결과를 날짜별로 Parquet 파일로 저장
# try:
#     for partition in query_job.result().to_dataframe_iterable():
#         if not partition.empty:
#             # 이벤트 날짜를 YYYY-mm-dd 형식으로 변환하여 파일명 생성
#             partition_date = partition['Event_Date'].iloc[0]
#             # 저장할 경로 확인 및 생성
#             save_path = f'../result/{partition_date}.parquet'
#             # os.makedirs(os.path.dirname(save_path), exist_ok=True)
#             # Parquet 파일로 저장
#             partition.to_parquet(save_path)
# except Exception as e:
#     print(f'error: {e}')


# Initialize the BigQuery client

data_by_date = {}

# Process each partition in the iterable
try:
    for partition in tqdm(query_job.result().to_dataframe_iterable()):
        if not partition.empty:
            for index, row in partition.iterrows():
                date_key = row['Event_Date']
                if date_key not in data_by_date:
                    data_by_date[date_key] = []
                data_by_date[date_key].append(row)

    # Now write each group to a Parquet file
    for date_key, rows in data_by_date.items():
        df = pd.DataFrame(rows)
        save_path = f'../result/{date_key}.parquet'
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        df.to_parquet(save_path, engine='pyarrow', compression='snappy', index=False)
except Exception as e:
    print(f'Error: {e}')


0it [00:00, ?it/s]

996it [33:08,  2.00s/it]


Error: malloc of size 1610612736 failed


In [7]:
pd.DataFrame()

""


In [12]:
pd.DataFrame(query_job)

,0
0,(61678d00-0f6c-4aaf-9750-e1467234cea7)
1,(55ce079c-b2b5-4b23-9b93-c9b3647e5061)
2,(d5804fd3-5355-49ce-aeda-7b9bbdacb13d)
3,(bd4fd3dd-d491-455b-8e53-db39b6effa74)
4,(51bde99e-99d0-46eb-9a88-31bac53e4265)
...,...
756,(ba141450-a8d4-427c-8032-ea7fcbeef1e8)
757,(8dc9ca7b-993e-413d-b39d-710b55508334)
758,(a7c7df98-2704-4a6a-8725-fa3359a8e875)
759,(5795eb60-f5cb-40ac-9190-d88b8c881744)


In [17]:
from google.cloud import bigquery
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import os
from tqdm import tqdm

# 결과를 날짜별로 Parquet 파일로 저장
try:
    for partition in tqdm(query_job.result().to_dataframe_iterable()):
        if not partition.empty:
            for index, row in partition.iterrows():
                # 이벤트 날짜를 YYYY-mm-dd 형식으로 변환하여 파일명 생성
                partition_date = row['Event_Date']
                save_path = f'../result/{partition_date}.parquet'
                # 파일 경로 확인 및 생성
                os.makedirs(os.path.dirname(save_path), exist_ok=True)
                
                # 데이터 프레임 생성, 인덱스 제거
                df = pd.DataFrame([row]).reset_index(drop=True)
                
                # 파일이 이미 존재하면 데이터를 읽어 추가 후 저장, 존재하지 않으면 새로 작성
                if os.path.exists(save_path):
                    # 기존 데이터 읽기
                    table_to_append = pq.read_table(save_path)
                    # 새 데이터를 PyArrow 테이블로 변환, 인덱스 제거
                    new_table = pa.Table.from_pandas(df, preserve_index=False)
                    # 데이터 합치기
                    combined_table = pa.concat_tables([table_to_append, new_table])
                    # 파일에 저장
                    pq.write_table(combined_table, save_path)
                else:
                    # 새 파일 작성
                    df.to_parquet(save_path, engine='pyarrow', compression='snappy', index=False)
except Exception as e:
    print(f'Error: {e}')


54it [13:48:48, 920.91s/it] 


KeyboardInterrupt: 

---

In [8]:
## 판다스 거치지 않은 더 빠른 쿼리
from google.cloud import bigquery
import pyarrow.parquet as pq
import pyarrow as pa
import os

# 결과를 날짜별로 Parquet 파일로 저장
try:
    # Prepare a writer to handle output stream
    writers = {}
    for row in query_job.result():
        # 이벤트 날짜를 YYYY-mm-dd 형식으로 변환하여 파일명 생성
        partition_date = row['Event_Date']
        save_path = f'../result/{partition_date}.parquet'

        # 파일 경로 확인 및 생성
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        
        if save_path not in writers:
            schema = pa.Table.from_pandas(pd.DataFrame([row])).schema
            writers[save_path] = pq.ParquetWriter(save_path, schema, compression='snappy')
        
        # Write row to the corresponding Parquet file
        table = pa.Table.from_pandas(pd.DataFrame([row]), schema=schema)
        writers[save_path].write_table(table)

    # Close all writers
    for writer in writers.values():
        writer.close()
except Exception as e:
    print(f'Error: {e}')

Error: ("Could not convert Row(('2024-04-24', '634b0a1e-bf91-49f1-940d-8266e3ab0a96', '워킹맨 아울렛 1+1 봄 여름 시즌 스판소재 런닝 자켓 아웃도어 외출복 일상복 빅사이즈 F101,지프 스피릿 남성 헨리넥 긴팔 티셔츠 305001,워킹맨 아울렛 1+1 봄 여름 시즌 후드탈부착형 경랑 라이트 메쉬 나일론 스판 바람막이 아웃도어 아우터 빅사이즈 F100,에버유어스 남성 바람막이 점퍼 아웃도어 등산복 후드 탈 부착자켓,곽씨네슈퍼 남성 골프 긴팔 티셔츠 골프복 골프웨어,에버유어스 아웃도어 봄 가을 남성 후드 탈부착 방풍방수 바람막이 자켓 등산복 재킷 일상복 상의 2color,레이지옴므 남성용 기본 무지 제인 집업가디건 간절기 니트 남자 카디건,[벤브로] 남자 봄 여름 가을 바람막이 경량 방수 후드자켓,언탭트 남성용 부드러운 워셔블 비스코스 니트 가디건,마운티스트 드라이 소프트 긴팔 집업 티셔츠 (남) 봄가을초겨울 따뜻한 부드러운,워킹맨 아울렛 4세트 봄 초여름 시즌 쾌적한 쿨론 집업 티셔츠 AS227,워킹맨 아울렛 1+1 봄 초여름 데일리 카라 긴팔티셔츠 작업복 일상복 근무복 아웃도어 티셔츠 AF216,워킹맨 아울렛 1+1 봄 가을 시즌 스판소재 남성 아웃도어 긴팔 스판 티셔츠 작업복 일상복 현장복 근무복 홈웨어 빅사이즈 AF205,봄 가을 시즌 남성 긴팔 집업 데일리 티셔츠 등산복 작업복 빅사이즈 스판티셔츠 A220,벤브로 남성용 데일리 라이더 자켓 캐주얼 가죽자켓+워런티 카드,워킹맨 아울렛 남성 1+1 봄 가을 시즌 스판 건빵 익스트림 펜츠 아웃도어 작업복 캠핑바지 낚시바지 일상복 BF128,샤필 에어포스 가을 겨울 항공점퍼 용 호랑이 잉어 남자 여 자수 점퍼 일본 스카쟌 남성 항공점퍼 누빔 가을 블루종 바람막이,영앤올드 겨울 남성 경량 웰론 패딩 점퍼 잠바 엠보 근무복 단체복 사무실 일상복,블랙야크 남여 공용 반팔티 B트리플라운드S 기능성 아이스 라운드 3팩 3종 세트', 'after'), 

In [11]:
from google.cloud import bigquery
import pyarrow.parquet as pq
import pyarrow as pa
import os
import pandas as pd

# Prepare a writer to handle output stream
writers = {}

# Define Arrow Schema explicitly based on your expected data types
schema = pa.schema([
    ('Event_Date', pa.string()),
    ('User_ID', pa.string()),
    ('product_names', pa.string()),
    ('period', pa.string())
])

try:
    for row in query_job.result():
        # Convert the Row to a dictionary first, then create a DataFrame
        row_dict = {
            'Event_Date': str(row['Event_Date']),
            'User_ID': str(row['User_ID']),
            'product_names': str(row['product_names']),
            'period': str(row['period'])
        }
        df = pd.DataFrame([row_dict])
        
        # 이벤트 날짜를 YYYY-mm-dd 형식으로 변환하여 파일명 생성
        partition_date = row['Event_Date']
        save_path = f'../result/{partition_date}.parquet'

        # 파일 경로 확인 및 생성
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        
        if save_path not in writers:
            writers[save_path] = pq.ParquetWriter(save_path, schema, compression='snappy')
        
        # Write row to the corresponding Parquet file
        table = pa.Table.from_pandas(df, schema=schema, preserve_index=False)
        writers[save_path].write_table(table)

    # Close all writers
    for writer in writers.values():
        writer.close()
except Exception as e:
    print(f'Error: {e}')


Error: 


In [12]:
path

'../result\\2024-04-12.parquet'

In [14]:
dir = os.listdir('../result')
path = os.path.join('../result',dir[3]) 


In [15]:
df = pd.read_parquet(path)

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [17]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.